In [2]:
import glob, json, os
from PIL import Image
from tqdm import tqdm_notebook
import numpy as np
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt

img_paths = glob.glob('./pics/*.jpg')[:10]  # 获取当前目录下前10张jpg图片
user_texts = ["一个主体通红的月亮", 
              "一只戴着黑框眼镜的小猫", 
              "一个卫星俯视角的台风影像", 
              "一条很长的大桥延伸至远处", 
              "一个光头男人穿着披风戴着红拳套", 
              "一个路标指向很多方向", 
              "一个夜幕下的赛博朋克风格城市", 
              "一个双手交叉于身前的笑着的男人", 
              "一个杂志封面的白衣浅色头发女人", 
              "一张讲解多模态的PPT"]


# 加载CLIP模型

In [3]:
from PIL import Image
import requests
from transformers import ChineseCLIPProcessor, ChineseCLIPModel
import torch

model = ChineseCLIPModel.from_pretrained("./model/chinese-clip-vit-base-patch16") # 中文clip模型
processor = ChineseCLIPProcessor.from_pretrained("./model/chinese-clip-vit-base-patch16") # 预处理

# 图像编码

In [4]:
img_image_feat = []
batch_size = 10
# 加载并预处理图像
imgs = []
for path in img_paths:
    img = Image.open(path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize((224, 224), Image.Resampling.LANCZOS)
    imgs.append(img)
# print(len(imgs))
# print(imgs[0].size)
inputs = processor(images=imgs, return_tensors="pt")
with torch.no_grad():
    image_features = model.get_image_features(**inputs)
    image_features = image_features.data.numpy()
    img_image_feat.append(image_features)

img_image_feat = np.vstack(img_image_feat)
img_image_feat = normalize(img_image_feat)

# 文本编码

In [5]:
img_texts_feat = []
texts = user_texts
inputs = processor(text=texts, return_tensors="pt", padding=True)
with torch.no_grad():
    text_features = model.get_text_features(**inputs)
    text_features = text_features.data.numpy()
    img_texts_feat.append(text_features)
    
img_texts_feat = np.vstack(img_texts_feat)
img_texts_feat = normalize(img_texts_feat)

# 计算相似度并匹配

In [6]:
results = []
for i in range(len(img_paths)):
    sim_result = np.dot(img_image_feat[i], img_texts_feat.T)
    best_match_idx = sim_result.argsort()[::-1][0]
    results.append((img_paths[i], user_texts[best_match_idx], sim_result[best_match_idx]))

In [7]:
print("图文匹配结果：")
for i, (img_path, matched_text, score) in enumerate(results):
    print(f"图片 {i+1}: {os.path.basename(img_path)}")
    print(f"匹配文本: {matched_text}")
    print(f"相似度: {score:.4f}")
    print("-" * 50)

图文匹配结果：
图片 1: 1.jpg
匹配文本: 一个光头男人穿着披风戴着红拳套
相似度: 0.4117
--------------------------------------------------
图片 2: 29536078457b9bb10410b70450dab975.jpg
匹配文本: 一只戴着黑框眼镜的小猫
相似度: 0.4632
--------------------------------------------------
图片 3: 2b6f97d54d00bcf1b0997d0781d06456.jpg
匹配文本: 一个夜幕下的赛博朋克风格城市
相似度: 0.4174
--------------------------------------------------
图片 4: 3b456b3bf3b9a252136fea29064cc8cb.jpg
匹配文本: 一个卫星俯视角的台风影像
相似度: 0.4882
--------------------------------------------------
图片 5: 516b9942f6000efa527025441cee376a.jpg
匹配文本: 一个路标指向很多方向
相似度: 0.4374
--------------------------------------------------
图片 6: db7b2fc020da5749f04e1a66931e36ff.jpg
匹配文本: 一个主体通红的月亮
相似度: 0.4694
--------------------------------------------------
图片 7: Snipaste_2025-08-03_13-54-44.jpg
匹配文本: 一个夜幕下的赛博朋克风格城市
相似度: 0.4236
--------------------------------------------------
图片 8: task02.jpg
匹配文本: 一张讲解多模态的PPT
相似度: 0.4710
--------------------------------------------------
图片 9: 微信图片_20250911121625_133_113.jpg
匹配文本: 一个杂志封面的白衣